In [1]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor_bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

predictor_bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import random
from checklist.editor import Editor
from checklist.perturb import Perturb

# Initialize the editor
editor = Editor()

# Define templates with varying structures and arguments
templates = [
    '{person} {verb} {obj}',
    '{person} {verb} {obj} {prep} {loc}',
    '{person} {verb} {obj} {conj} {verb2} {obj2}',
    '{person} {verb} {obj} {prep} {loc} {conj} {verb2} {obj2}'
]

# Helper function to get random elements from a list
def random_choice(elements):
    return random.choice(elements)

# Generate sentences with random combinations of words for each template
sentences = []
true_arguments = []

for template in templates:
    for _ in range(10):  # Generate 10 sentences for each template
        sentence = editor.template(template,
            person=random_choice(editor.lexicons['person']),
            verb=random_choice(editor.lexicons['verb']),
            obj=random_choice(editor.lexicons['noun']),
            prep=random_choice(editor.lexicons['prep']),
            loc=random_choice(editor.lexicons['location']),
            conj=random_choice(editor.lexicons['conj']),
            verb2=random_choice(editor.lexicons['verb']),
            obj2=random_choice(editor.lexicons['noun'])
        )
        
        # Get the indices of arguments in the sentence
        words = sentence.split()
        args_indices = [i for i, w in enumerate(words) if w in editor.lexicons]
        
        sentences.append(sentence)
        true_arguments.append(args_indices)

# Perturb the sentences to add additional variability
perturbed_sentences, perturbed_true_arguments = Perturb.perturb(sentences, true_arguments, nsamples=5, meta=True)

# Print some examples
for i in range(5):
    print(f"Sentence: {perturbed_sentences[i]}")
    print(f"True arguments: {perturbed_true_arguments[i]}\n")

KeyError: 'person'

In [3]:
editor.